This script is used for:   
- creating and applying the urban mask to CAM data   
- merging CAM and CLM (urban) data as a `xarray Dataset`   

In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import cftime
import gc
import time
from tqdm import tqdm

save_path = "/glade/scratch/zhonghua/urban_params/urban_LE/"


def get_ts_masked_cam(start_date, end_date, var, member, CLM_lat, mask):   
    # read raw data
    var_raw = xr.open_dataset("/glade/collections/cdg/data/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/daily/" \
                              + var + "/b.e11.BRCP85C5CNBDRD.f09_g16."\
                              + member +".cam.h1."\
                              + var + ".20060101-20801231.nc")
    # select time period and reset coordinate index based on CLM
    var_reg = var_raw[var].sel(time=slice(start_date, end_date)).assign_coords(lat = CLM_lat)
    # apply the mask 
    var_reg_mask = var_reg.where(mask)
    # rename the DataArray
    var_final = var_reg_mask.rename(var)
    return var_final  

# full pipeline
def get_urban_ds(start_date, end_date, member):
    print("start_date:", start_date)
    print("end_date:", end_date) 
    print("member:", member)
    
    # ====================
    start_time = time.time()
    # load the urban maximal temperature
    TREFMXAV_U = xr.open_dataset("/glade/collections/cdg/data/cesmLE/CESM-CAM5-BGC-LE/lnd/proc/tseries/daily/"\
                                 +"TREFMXAV_U/b.e11.BRCP85C5CNBDRD.f09_g16."\
                                 + member + ".clm2.h1."\
                                 + "TREFMXAV_U.20060101-20801231.nc")
    # get the clm latitude
    CLM_lat = TREFMXAV_U.indexes['lat']
    # get the mask of clm
    mask = TREFMXAV_U["TREFMXAV_U"].loc["2006-01-02"].notnull().squeeze()
    # get the Urban maximal temperature
    U_max = TREFMXAV_U["TREFMXAV_U"].sel(time=slice(start_date, end_date))
    
    # create a list to loop the variables
    temp = [U_max]
    var_list = ["FLNS","FSNS",
                "PRECSC","PRECSL","PRECT",
                "QBOT","TREFHT","UBOT","VBOT"]

    for var in var_list:
        df = get_ts_masked_cam(start_date, end_date, 
                               var, member, CLM_lat, mask)
        temp.append(df)
        del df
        gc.collect()
    print("It took", time.time()-start_time,"to read CLM and CAM data and apply urban mask")
    # ====================
    
    # ====================
    # merge the list as a Xarray Dataset
    start_time = time.time()
    ds = xr.merge(temp)
    print("It took", time.time()-start_time,"to merge CLM and CAM data")
    # ====================
    
    # ====================
    # save the xarray dataset
    start_time = time.time()
    ds["PRSN"] = ds["PRECSC"] + ds["PRECSL"]
    final_var_list = ["TREFMXAV_U",
                      "FLNS","FSNS",
                      "PRECT","PRSN",
                      "QBOT","TREFHT",
                      "UBOT","VBOT"]
    ds[final_var_list].to_netcdf(save_path + member\
                 + "_"+start_date[:4] + "_" + end_date[:4] + ".nc")
    print("It took", time.time()-start_time,"to save data as a netcdf file")
    print("\n")
    # ====================

In [2]:
%%time
for i in tqdm(range(2,34)):
    member = (str(i).zfill(3))
    get_urban_ds(start_date="2006-01-02", end_date="2015-12-31", member=member)
    get_urban_ds(start_date="2061-01-01", end_date="2070-12-31", member=member) 

  0%|                                                                                                                                                | 0/32 [00:00<?, ?it/s]

start_date: 2006-01-02
end_date: 2015-12-31
member: 002
It took 42.054776430130005 to read CLM and CAM data and apply urban mask
It took 0.03783226013183594 to merge CLM and CAM data
It took 12.002175569534302 to save data as a netcdf file


start_date: 2061-01-01
end_date: 2070-12-31
member: 002
It took 40.77051329612732 to read CLM and CAM data and apply urban mask
It took 0.03709864616394043 to merge CLM and CAM data


  3%|████▏                                                                                                                                  | 1/32 [01:47<55:32, 107.49s/it]

It took 12.538486242294312 to save data as a netcdf file


start_date: 2006-01-02
end_date: 2015-12-31
member: 003
It took 39.690590381622314 to read CLM and CAM data and apply urban mask
It took 0.03629612922668457 to merge CLM and CAM data
It took 11.994596481323242 to save data as a netcdf file


start_date: 2061-01-01
end_date: 2070-12-31
member: 003
It took 41.46476697921753 to read CLM and CAM data and apply urban mask
It took 0.036382436752319336 to merge CLM and CAM data


  6%|████████▍                                                                                                                              | 2/32 [03:35<54:01, 108.05s/it]

...

It took 15.069166421890259 to save data as a netcdf file


start_date: 2006-01-02
end_date: 2015-12-31
member: 032
It took 22.504070520401 to read CLM and CAM data and apply urban mask
It took 0.0364072322845459 to merge CLM and CAM data
It took 13.828955888748169 to save data as a netcdf file


start_date: 2061-01-01
end_date: 2070-12-31
member: 032
It took 23.810172080993652 to read CLM and CAM data and apply urban mask
It took 0.03769230842590332 to merge CLM and CAM data


 97%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊    | 31/32 [57:20<01:33, 93.82s/it]

It took 15.929891109466553 to save data as a netcdf file


start_date: 2006-01-02
end_date: 2015-12-31
member: 033
It took 21.785003662109375 to read CLM and CAM data and apply urban mask
It took 0.03648519515991211 to merge CLM and CAM data
It took 15.261108875274658 to save data as a netcdf file


start_date: 2061-01-01
end_date: 2070-12-31
member: 033
It took 20.51813817024231 to read CLM and CAM data and apply urban mask
It took 0.037122249603271484 to merge CLM and CAM data


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [58:35<00:00, 109.85s/it]

It took 16.9983971118927 to save data as a netcdf file


CPU times: user 42min 5s, sys: 8min 45s, total: 50min 51s
Wall time: 58min 35s
